<a href="https://colab.research.google.com/github/miinsss/Capstone-Project/blob/main/04_contract_data_models_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libaries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

from sklearn.impute import SimpleImputer

from imblearn.over_sampling import SMOTE

# Load the Contract Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load the contract_info.csv file
contract_info_path = '/content/drive/My Drive/Capstone/Data/Categories/contract_info.csv'
contract_info = pd.read_csv(contract_info_path)

# Display the first few rows to verify content
print(contract_info.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
    ANO_SID CORPORATE_DEVISION   ORTPLZ          ORTS-NAME            STRASSE  \
0  43597657                VGV  50769.0               Köln         Gohrer Weg   
1  29261516                VGV  51427.0  Bergisch Gladbach     Buchenkampsweg   
2   4118112                VGV  42553.0            Velbert   Elberfelder Str.   
3  41087246                W&W  51147.0               Köln        Barbarastr.   
4  32662333                VHV  42655.0           Solingen  Hossenhauser Str.   

   SUM_INSURED CONSTRACTION_DESIGN  CONSTRUCTION_YEAR        WFL ZONE  \
0     298800.0      NORMAL_VENTURE        1967.565648  106.24368  NaN   
1    1145400.0      NORMAL_VENTURE        1996.000000  106.24368  NaN   
2    2961000.0      NORMAL_VENTURE        1981.000000  106.24368  NaN   
3      32500.0                 NaN        1967.565648   50.00000    E   
4      54800.0     

# Linear Regression Model

## Preprocess the Data

In [ ]:
# Convert categorical variables to numeric using one-hot encoding
contract_info_encoded = pd.get_dummies(contract_info, drop_first=True)

# Define the features (X) and the target (y)
X = contract_info_encoded.drop(columns=['PRODUCTLINE_Kompakt'])  # Adjust target variable accordingly
y = contract_info_encoded['PRODUCTLINE_Kompakt']

## Select a subset

In [ ]:
# Select a subset of important features first
initial_features = ['SUM_INSURED', 'CONSTRUCTION_YEAR', 'WFL', 'PREMIUM_AMOUNT', 'CORPORATE_DEVISION_VGV', 'CORPORATE_DEVISION_VHV', 'CORPORATE_DEVISION_W&W']

# Ensure we only select these features if they exist in the dataset
initial_features = [feature for feature in initial_features if feature in X.columns]
X = X[initial_features]

## Split the Data

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Select Important Features

In [ ]:
# Impute missing values with the mean for numerical columns
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Optimazation
# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_imputed, y_train)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Initialize RandomForest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)

# Get feature importances
importances = rf.feature_importances_
feature_names = X.columns

# Create a DataFrame for feature importances
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Select top 5 features (adjust the number of features as needed)
top_features = feature_importances.head(5)['Feature'].values
X_train_top = X_train_scaled[:, [list(X.columns).index(feature) for feature in top_features]]
X_test_top = X_test_scaled[:, [list(X.columns).index(feature) for feature in top_features]]

## Train the Model

In [ ]:
# Initialize the Logistic Regression model
# log_reg = LogisticRegression(max_iter=1000)
# damit acc von ca 78%
log_reg = LogisticRegression(max_iter=1000, class_weight='balanced')

# Train the model on the training data with top features
log_reg.fit(X_train_top, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

## Evaluate the Model

In [ ]:
# Predict on the test data
y_pred = log_reg.predict(X_test_top)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Display the classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.5861305791220952
              precision    recall  f1-score   support

       False       0.99      0.47      0.64      6329
        True       0.35      0.98      0.51      1804

    accuracy                           0.59      8133
   macro avg       0.67      0.73      0.58      8133
weighted avg       0.85      0.59      0.61      8133

